In [18]:
import numpy as np

In [ ]:
# to make this notebook's output stable across runs
def reset_graph(seed=42):
    tf.reset_default_graph()
    tf.set_random_seed(seed)
    np.random.seed(seed)

1) Após termos discutido sobre o Tensorflow, em sua opinião, quais são as principais vantagens da criação de um grafo de computação, em vez de executar diretamente os cálculos? Quais são as principais desvantagens?

#### Principais benefícios:
- O TensorFlow pode calcular automaticamente os gradientes para você (usando reverse-mode autodiff).
- O TensorFlow pode executar as operações em paralelo em diferentes threads.
- Isso facilita a execução do mesmo modelo em diferentes dispositivos.
- Simplifica a introspecção - por exemplo, para visualizar o modelo no TensorBoard.

####  Principais desvantagens:
- Torna a curva de aprendizado mais íngreme.
- Torna a depuração passo a passo mais difícil

2) O comando a_val = a.eval(session=sess) é equivalente a a_val = sess.run(a)?

In [20]:
import tensorflow as tf

# Creating the graph.
c = tf.Variable(3, name="c")
a = c*c

# Executing the calculation graph.
sess = tf.Session()
sess.run(c.initializer)
a_val0 = a.eval(session=sess)

print(a_val0)


reset_graph()

# Creating the graph.
c = tf.Variable(4, name="c")
a = c*c

# Executing the calculation graph.
sess = tf.Session()
sess.run(c.initializer)

a_val1 = sess.run(a)

print(a_val1)

sess.close()

9
16


Sim, conforme pode-se ver acima, a instrução a_val = a.eval(session=sess) é equivalente a a_val = sess.run(a).

3) O comando a_val, b_val = a.eval(session=sess), b.eval (session=sess) é equivalente a a_val, b_val = sess.run([a, b])?

In [21]:
# Creating the graph.
a = tf.Variable(33, name="a")
b = tf.Variable(66, name="b")

# Executing the calculation graph.
sess = tf.Session()
sess.run(a.initializer)
sess.run(b.initializer)

a_val, b_val = a.eval(session=sess), b.eval (session=sess)

print(a_val)
print(b_val)

a_val1, b_val1 = sess.run([a, b])
print(a_val1)
print(b_val1)

sess.close()

33
66
33
66


Não, o comando a_val, b_val = a.eval(session=sess), b.eval(session=sess) não é equivalente a a_val, b_val = sess.run([a, b]). Na verdade, o primeiro comando executa o grafo duas vezes (uma vez para calcular a, e outra vez para calcular b), enquanto o segundo comando executa o grafo apenas uma vez. Se alguma dessas operações (ou as operações das quais elas dependem) tiverem efeitos colaterais (por exemplo, uma variável for modificada, um item for inserido em uma fila ou um leitor ler um arquivo), os efeitos serão diferentes. Se eles não tiverem efeitos colaterais, as duas instruções retornarão o mesmo resultado, mas a segunda será mais rápida que a primeira.

4) É possível se executar dois grafos na mesma sessão?

Não, não é possívelse executar dois grafos na mesma sessão. Para que isso fosse possível, primeiro, seria necessário unir os 2 grafos em um único grafo.

5) Quando uma variável é inicializada? Quando ela é destruída?

Uma variável é inicializada quando você chama seu inicializador e é destruída quando a sessão termina. No TensorFlow distribuído, as variáveis ficam em contêineres no cluster, portanto, fechar uma sessão não destruirá a variável. Para destruir uma variável, você precisa limpar seu contêiner.

6) Qual é a diferença entre um nó do tipo placeholder e um nó do tipo variável?

Variáveis e placeholders são extremamente diferentes:

• Uma variável é uma operação que possui um valor. Se você executar a variável, ela retornará esse valor. Antes de poder executá-la, é necessário inicializá-la. Você pode alterar o valor da variável (por exemplo, usando uma operação de atribuição). As variáveis são stateful, ou seja, elas mantém o mesmo valor após execuções sucessivas do grafo. Elas são normalmente usadas para armazenar parâmetros do modelo, mas também para outros fins (por exemplo, para contar os passos de treinamento).

• Os placeholders tecnicamente não fazem muito: eles apenas mantêm informações sobre o tipo e a forma do tensor que representam, mas não têm valor. De fato, se você tentar avaliar uma operação que depende de um placeholder, você deve alimentar o TensorFlow com o valor do placeholder (usando o argumento feed_dict) ou obterá uma exceção. Os placeholders são normalmente usados para alimentar dados de treinamento ou teste ao grafo do TensorFlow durante a fase de execução. Eles também são úteis para passar um valor para um nó de atribuição, para alterar o valor de uma variável (por exemplo, pesos do modelo).

7) O que acontece quando você executa um grafo para avaliar uma operação que depende de um placeholder, mas você não fornece nenhum valor para o placeholder? O que acontece se a operação não depender do placeholder?

In [30]:
# Creating the graph.
x = tf.placeholder(tf.int32, shape=(1,), name="x")
y = tf.Variable(4, name="y")
f = x*x*y + y + 2

init = tf.global_variables_initializer()

# Executing the calculation graph.
with tf.Session() as sess:
    init.run()
    result = f.eval()

print(result)

sess.close()

InvalidArgumentError: You must feed a value for placeholder tensor 'x_8' with dtype int32 and shape [1]
	 [[node x_8 (defined at <ipython-input-30-9f5183753afa>:2) ]]

Original stack trace for 'x_8':
  File "C:\ProgramData\Anaconda3\lib\runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "C:\ProgramData\Anaconda3\lib\runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance()
  File "C:\ProgramData\Anaconda3\lib\site-packages\traitlets\config\application.py", line 658, in launch_instance
    app.start()
  File "C:\ProgramData\Anaconda3\lib\site-packages\ipykernel\kernelapp.py", line 505, in start
    self.io_loop.start()
  File "C:\ProgramData\Anaconda3\lib\site-packages\tornado\platform\asyncio.py", line 148, in start
    self.asyncio_loop.run_forever()
  File "C:\ProgramData\Anaconda3\lib\asyncio\base_events.py", line 539, in run_fore

In [39]:
reset_graph()

# Creating the graph.
x = tf.placeholder(tf.int32, shape=(), name="x")
y = tf.Variable(4, name="y")
f = y*y*y + y + 2

init = tf.global_variables_initializer()

# Executing the calculation graph.
with tf.Session() as sess:
    init.run()
    result = f.eval(feed_dict={x:3})

print(result)

sess.close()

70


Se você executar o grafo para avaliar uma operação que depende de um placeholder, mas não fornece seu valor, você recebe uma exceção. Se a operação não depender do placeholder, nenhuma exceção será gerada.

8) Como você pode definir o valor de uma variável com qualquer valor desejado durante a fase de execução? Escreva um pequeno trecho de código para exemplificar.

Você pode especificar o valor inicial de uma variável ao construir o grafo e ela será inicializada mais tarde quando você executar o inicializador da variável durante a fase de execução. Se você deseja alterar o valor dessa variável com o valor que desejar durante a fase de execução, a opção mais simples é criar um nó de atribuição (durante a fase de construção do grafo) usando a função tf.assign(), passando a variável e um placeholder como parâmetros. Durante a fase de execução, você pode executar a operação de atribuição e alimentar o novo valor da variável usando o placeholder.

In [49]:
x = tf.Variable(2.1)
x_new_val = tf.placeholder(shape=(), dtype=tf.float32)
x_assign = x.assign(x_new_val)

with tf.Session():
    x.initializer.run() 
    print(x.eval()) # 2.1
    x_assign.eval(feed_dict={x_new_val: 5.6})
    print(x.eval()) # 5.0

2.1
5.6
